In [ ]:
#| default_exp base

In [ ]:
from nbdev import *

# Base

> Module for making requests to ISTAT API

In [ ]:
#| export
import requests

In [ ]:
#| export
class ISTAT:
    """Base class that provides useful functions to communicate with ISTAT API"""

    def __init__(self):
        self.base_url = "http://sdmx.istat.it/SDMXWS/rest"
        self.agencyID = "IT1"

    def _request(self, path, **kwargs):
        """Make a request to ISTAT API given a 'path'"""
        url = "/".join([self.base_url, path])

        if "headers" in kwargs.keys():
            response = requests.get(url, headers=kwargs["headers"])
        else:
            response = requests.get(url)

        return response

In [ ]:
show_doc(ISTAT._request)

<h4 id="ISTAT._request" class="doc_header"><code>ISTAT._request</code><a href="__main__.py#L9" class="source_link" style="float:right">[source]</a></h4>

> <code>ISTAT._request</code>(**`path`**, **\*\*`kwargs`**)

Make a request to ISTAT API given a 'path'

The 'ISTAT' class already implements the 'base_url', therefore 'path' only needs to include the other parameters of the request. Here is an example of a succesful request:

In [ ]:
istat = ISTAT()
path = "dataflow/IT1"
test_response = istat._request(path=path)
test_response

<Response [200]>

In [ ]:
#| hide
export.nbdev_export()

Converted 00_base.ipynb.
Converted 01_discovery.ipynb.
Converted 02_utils.ipynb.
Converted 03_retrieval.ipynb.
Converted index.ipynb.
